***Correlating screen time with other factors***

In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from itertools import combinations #will told me about this
%matplotlib inline

In [3]:
#create file
d = pd.read_csv("dummy_data.csv")
df = d[["time_spent", "age", "gender", "interests", "location", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]]
df

,time_spent,age,gender,interests,location,demographics,profession,income,indebt,isHomeOwner,Owns_Car
0,3,56,male,Sports,United Kingdom,Urban,Software Engineer,19774,True,False,False
1,2,46,female,Travel,United Kingdom,Urban,Student,10564,True,True,True
2,8,32,male,Sports,Australia,Sub_Urban,Marketer Manager,13258,False,False,False
3,5,60,non-binary,Travel,United Kingdom,Urban,Student,12500,False,True,False
4,1,25,male,Lifestlye,Australia,Urban,Software Engineer,14566,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...
995,8,22,female,Lifestlye,United Kingdom,Rural,Marketer Manager,18536,False,True,False
996,6,40,non-binary,Travel,United Kingdom,Rural,Software Engineer,12711,True,False,False
997,5,27,non-binary,Travel,United Kingdom,Rural,Student,17595,True,False,True
998,4,61,female,Sports,Australia,Sub_Urban,Marketer Manager,16273,True,True,False


In [3]:
#clean data
df["gender"] = preprocessing.LabelEncoder().fit(["male", "female", "non-binary"]).transform(df["gender"])
df["interests"] = preprocessing.LabelEncoder().fit(["Sports", "Travel", "Lifestlye"]).transform(df["interests"])
df["location"] = preprocessing.LabelEncoder().fit(["United States", "United Kingdom", "Australia"]).transform(df["location"])
df["demographics"] = preprocessing.LabelEncoder().fit(["Urban", "Sub_Urban", "Rural"]).transform(df["demographics"])
df["profession"] = preprocessing.LabelEncoder().fit(["Student", "Software Engineer", "Marketer Manager"]).transform(df["profession"]) #these throw harmless warnings.
df["indebt"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["indebt"])
df["isHomeOwner"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["isHomeOwner"])
df["Owns_Car"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["Owns_Car"])
X = df[["time_spent", "age", "gender", "interests", "location", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]].values.astype(float) 

/var/folders/np/wt2f_g3s42g_nygnmnbpcswr0000gn/T/ipykernel_32564/2201030753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["gender"] = preprocessing.LabelEncoder().fit(["male", "female", "non-binary"]).transform(df["gender"])
/var/folders/np/wt2f_g3s42g_nygnmnbpcswr0000gn/T/ipykernel_32564/2201030753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["interests"] = preprocessing.LabelEncoder().fit(["Sports", "Travel", "Lifestlye"]).transform(df["interests"])
/var/folders/np/wt2f_g3s42g_nygnmnbpc

In [4]:
#ignore the above warnings. They are harmless and not simple to fix

In [5]:
#make variable arrays
features = ["age", "gender", "interests", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]
ind = df[features]
deps = df[["time_spent"]]

In [6]:
#normalize data
ind = pd.DataFrame(preprocessing.StandardScaler().fit(ind).transform(ind))
ind.columns = features


In [7]:
#inspect data
ind.head()

,age,gender,interests,demographics,profession,income,indebt,isHomeOwner,Owns_Car
0,1.112882,-0.001228,0.015896,1.244883,0.056541,1.609380,1.006018,-1.016130,-1.081294
1,0.371652,-1.229356,1.238658,1.244883,1.285703,-1.505106,1.006018,0.984126,0.924818
2,-0.666069,-0.001228,0.015896,0.018397,-1.172620,-0.594094,-0.994018,-1.016130,-1.081294
3,1.409373,1.226900,1.238658,1.244883,1.285703,-0.850422,-0.994018,0.984126,-1.081294
4,-1.184929,-0.001228,-1.206866,1.244883,0.056541,-0.151776,-0.994018,0.984126,0.924818


In [8]:
#create a list of all possible combinations of features
feature_combs = []
for length in range(1, len(features) + 1):
    feature_combs.extend(list(combinations(features, length)))

In [9]:
len(feature_combs)

511

In [10]:
acc_list = []
for comb in feature_combs:
    comb_ind = ind[list(comb)]
    train_d, test_d, train_i, test_i = train_test_split(deps, comb_ind, test_size=0.2, random_state=4) #split data
    for k in range(20, 200): #this loop finds the the accuracy for 100<k<300
        m = KNeighborsClassifier(n_neighbors = k, n_jobs = -1).fit(train_i, np.ravel(train_d)) #fits a model for k = k
        yhat = m.predict(test_i)  #generates predictions for model m on test set
        acc = metrics.accuracy_score(test_d, yhat) #gets accuracy score on test set
        acc_list.append([acc, k])

In [11]:
max(acc_list)

[0.23, 55]

In [12]:
# this is 23% accuracy--better than random (11.1%), but clearly not very good. Let's try if it can predict whether it will be less than or greater than some value (binary).

In [13]:
print(deps.head)
new_deps = []
for cat in deps.time_spent:
    if int(cat) > 4: new_deps.append(1)
    else: new_deps.append(0)
new_deps = np.asarray(new_deps)
new_deps[0:5]

<bound method NDFrame.head of      time_spent
0             3
1             2
2             8
3             5
4             1
..          ...
995           8
996           6
997           5
998           4
999           8

[1000 rows x 1 columns]>


array([0, 0, 1, 1, 0])

In [18]:
acc_list2 = []

train_d, test_d, train_i, test_i = train_test_split(new_deps, comb_ind, test_size=0.2, random_state=4) #split data
for k in range(20, 200): #this loop finds the the accuracy for 100<k<300
    m = KNeighborsClassifier(n_neighbors = k, n_jobs = -1).fit(train_i, np.ravel(train_d)) #fits a model for k = k
    yhat = m.predict(test_i)  #generates predictions for model m on test set
    acc = metrics.accuracy_score(test_d, yhat) #gets accuracy score on test set
    acc_list2.append([acc, k])

In [19]:
max(acc_list2)

[0.565, 82]

In [ ]:
#only slightly better than average. Let's try Logistic Regression next